<h1>European Flights Analysis<h1>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px


df = pd.read_csv("data/flights.csv")
df

,YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_ICAO,APT_NAME,STATE_NAME,FLT_DEP_1,FLT_ARR_1,FLT_TOT_1,FLT_DEP_IFR_2,FLT_ARR_IFR_2,FLT_TOT_IFR_2,Pivot Label
0,2016,1,JAN,2016-01-01T00:00:00Z,EBAW,Antwerp,Belgium,4,3,7,NaN,NaN,NaN,Antwerp (EBAW)
1,2016,1,JAN,2016-01-01T00:00:00Z,EBBR,Brussels,Belgium,174,171,345,174.0,161.0,335.0,Brussels (EBBR)
2,2016,1,JAN,2016-01-01T00:00:00Z,EBCI,Charleroi,Belgium,45,47,92,45.0,45.0,90.0,Charleroi (EBCI)
3,2016,1,JAN,2016-01-01T00:00:00Z,EBLG,Liège,Belgium,6,7,13,NaN,NaN,NaN,Liège (EBLG)
4,2016,1,JAN,2016-01-01T00:00:00Z,EBOS,Ostend-Bruges,Belgium,7,7,14,NaN,NaN,NaN,Ostend-Bruges (EBOS)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688094,2022,5,MAY,2022-05-31T00:00:00Z,LYBE,Belgrade - Nikola Tesla,Serbia,80,85,165,NaN,NaN,NaN,Belgrade - Nikola Tesla (LYBE)
688095,2022,5,MAY,2022-05-31T00:00:00Z,LYPG,Podgorica,Montenegro,19,18,37,NaN,NaN,NaN,Podgorica (LYPG)
688096,2022,5,MAY,2022-05-31T00:00:00Z,LZIB,Bratislava,Slovakia,21,21,42,20.0,19.0,39.0,Bratislava (LZIB)
688097,2022,5,MAY,2022-05-31T00:00:00Z,UDYZ,Yerevan,Armenia,39,40,79,NaN,NaN,NaN,Yerevan (UDYZ)


<h2>Test Graph</h2>

In [2]:
# Conta il numero di voli per ogni stato e ordina dal più grande al più piccolo
df_sorted = df["STATE_NAME"].value_counts().reset_index()
df_sorted.columns = ["State", "NumFlights"]
df_sorted = df_sorted.sort_values(by="NumFlights", ascending=True)  # Ordinamento decrescente

# Creazione dell'istogramma ordinato
fig = px.histogram(df_sorted, x="NumFlights", y="State", nbins=1000, title="Num of Flights per State (Ordered)")

# Mantieni l'ordine corretto assegnando i valori di Y manualmente
fig.update_layout(
    height=1200,  # Ingrandisci il grafico verticalmente
    yaxis=dict(categoryorder="array", categoryarray=df_sorted["State"].tolist())  # Ordine manuale
)

# Mostra il grafico
fig.show()


In [3]:

# Conversione della data
df["FLT_DATE"] = pd.to_datetime(df["FLT_DATE"])

# Calcolo media voli giornalieri per mese
monthly_avg = df.groupby("MONTH_NUM")["FLT_TOT_1"].mean().reset_index()

# Aggiunta nomi mesi
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
monthly_avg["Month"] = monthly_avg["MONTH_NUM"].apply(lambda x: month_names[x - 1])

# Ordinamento corretto
monthly_avg = monthly_avg.sort_values("MONTH_NUM")

# Grafico con Plotly
fig = px.bar(
    monthly_avg,
    x="Month",
    y="FLT_TOT_1",
    title="Media dei voli giornalieri per mese (2016–2022)",
    labels={"FLT_TOT_1": "Voli giornalieri medi", "Month": "Mese"}
)

fig.update_layout(xaxis_title="Mese", yaxis_title="Voli giornalieri medi")
fig.show()


In [23]:
# Prepara il datetime e la colonna year-month
df["FLT_DATE"] = pd.to_datetime(df["FLT_DATE"])
df["YEAR_MONTH"] = df["FLT_DATE"].dt.to_period("M").dt.to_timestamp()

# Calcolo della media mensile dei voli giornalieri
monthly_avg = df.groupby("YEAR_MONTH")["FLT_TOT_1"].mean().reset_index()

# Estrai il mese e l'anno per le etichette
monthly_avg["MONTH"] = monthly_avg["YEAR_MONTH"].dt.month
monthly_avg["YEAR"]  = monthly_avg["YEAR_MONTH"].dt.year

# Inizializza le etichette a stringa vuota
monthly_avg["TICK_LABEL"] = ""

# Solo Gennaio: "Gen YYYY"
monthly_avg.loc[monthly_avg["MONTH"] == 1, "TICK_LABEL"] = \
    monthly_avg.loc[monthly_avg["MONTH"] == 1, "YEAR_MONTH"].dt.strftime("Gen %Y")

# Solo Maggio: "Mag"
monthly_avg.loc[monthly_avg["MONTH"] == 5, "TICK_LABEL"] = "Mag"

# Solo Settembre: "Set"
monthly_avg.loc[monthly_avg["MONTH"] == 9, "TICK_LABEL"] = "Set"

# Definisci i tick da mostrare
mask_ticks  = monthly_avg["MONTH"].isin([1, 5, 9])
tick_vals   = monthly_avg.loc[mask_ticks, "YEAR_MONTH"]
tick_texts  = monthly_avg.loc[mask_ticks, "TICK_LABEL"]

# Costruisci il grafico
fig = px.line(
    monthly_avg,
    x="YEAR_MONTH",
    y="FLT_TOT_1",
    markers=True,
    #title="Trend dei Voli Medi Giornalieri per Mese (2016–2022)",
    labels={"YEAR_MONTH": "Periodo (Mese)", "FLT_TOT_1": "Voli Medi Giornalieri"}
)

# Applica le etichette personalizzate solo a Gen, Mag e Set
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=tick_vals,
        ticktext=tick_texts,
        tickangle=-45,
        showgrid=True
    ),
    height=600
)

fig.show()

C:\Users\Christian\AppData\Local\Temp\ipykernel_34076\330044476.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [21]:
# Raggruppa per aeroporto e somma FLT_TOT_1 (totale voli)
arrivals = (
    df
    .groupby(["APT_ICAO", "APT_NAME"])["FLT_TOT_1"]
    .sum()
    .reset_index()
    .rename(columns={"FLT_TOT_1": "TOT_VOLI"})
)

# Seleziona i top 10 aeroporti per TOT_VOLI
top10 = arrivals.sort_values("TOT_VOLI", ascending=False).head(10)

# Calcola la percentuale sul totale del dataset
total_all = arrivals["TOT_VOLI"].sum()
top10["PERCENT"] = top10["TOT_VOLI"] / total_all * 100

# Ordina in ordine crescente in modo che la barra più lunga finisca in alto
top10 = top10.sort_values("TOT_VOLI", ascending=True)

# Crea il grafico a barre orizzontali
fig = px.bar(
    top10,
    x="TOT_VOLI",
    y="APT_NAME",
    orientation="h",
    #title="Top 10 aeroporti per traffico totale",
    labels={
        "TOT_VOLI": "Totale voli (arrivi + partenze)",
        "APT_NAME": "Aeroporto"
    },
    text="PERCENT"
)

# Formatta il testo come percentuale con un decimale
fig.update_traces(
    texttemplate="%{text:.1f}%",
    textposition="outside",
    marker_line_width=1.5
)

# Imposta l'asse X sui valori assoluti con separatore delle migliaia
fig.update_layout(
    xaxis=dict(
        tickformat=",",
        title="Numero totale di voli"
    ),
    margin=dict(l=200, r=40, t=70, b=60),
    height=600,
    yaxis=dict(categoryorder="total ascending")
)

fig.show()

In [22]:
# Raggruppa per aeroporto e somma FLT_TOT_1 (totale voli)
arrivals = (
    df
    .groupby(["APT_ICAO", "APT_NAME"])["FLT_TOT_1"]
    .sum()
    .reset_index()
    .rename(columns={"FLT_TOT_1": "TOT_VOLI"})
)

# Seleziona gli aeroporti con meno traffico (gli "ultimi 10")
bottom10 = arrivals.sort_values("TOT_VOLI", ascending=True).head(10)

# Calcola la percentuale sul totale del dataset
total_all = arrivals["TOT_VOLI"].sum()
bottom10["PERCENT"] = bottom10["TOT_VOLI"] / total_all * 100

# Per la visualizzazione orizzontale: il più grosso di questi 10 in alto
bottom10 = bottom10.sort_values("TOT_VOLI", ascending=True)

# Crea il grafico a barre orizzontali
fig = px.bar(
    bottom10,
    x="TOT_VOLI",
    y="APT_NAME",
    orientation="h",
    #title="Bottom 10 aeroporti per traffico totale (arrivi + partenze)",
    labels={
        "TOT_VOLI": "Totale voli (arrivi + partenze)",
        "APT_NAME": "Aeroporto"
    },
    text="PERCENT"
)

# Mostra il testo in formato percentuale con due decimali
fig.update_traces(
    texttemplate="%{text:.5f}%",
    textposition="outside",
    marker_line_width=1.5
)

# Imposta l'asse X sui valori assoluti con separatore delle migliaia
fig.update_layout(
    xaxis=dict(
        tickformat=",",
        title="Numero totale di voli"
    ),
    margin=dict(l=200, r=40, t=70, b=60),
    height=600,
    yaxis=dict(categoryorder="total ascending")
)

fig.show()

In [6]:
# Calcola il totale dei voli in partenza per aeroporto
departures = df.groupby(["APT_ICAO", "APT_NAME"])["FLT_DEP_1"].sum().reset_index()

# Seleziona i primi 10 aeroporti
top10_departures = departures.sort_values("FLT_DEP_1", ascending=False).head(10)

# Crea il grafico a barre
fig = px.bar(
    top10_departures,
    x="APT_NAME",
    y="FLT_DEP_1",
    title="Top 10 aeroporti per numero totale di voli in partenza",
    labels={"APT_NAME": "Aeroporto", "FLT_DEP_1": "Totale voli in partenza"},
    text="FLT_DEP_1"
)


fig.update_layout(
    xaxis_tickangle=30,
    yaxis_title="Totale voli in partenza",
    xaxis_title="Aeroporto",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis=dict(
        tickformat=",",
        nticks=10
    ),
    margin=dict(t=70, b=120, l=60, r=40),
    height=600
)

fig.show()

In [7]:
# Calcola arrivi e partenze totali
totali = df.groupby(["APT_ICAO", "APT_NAME"])[["FLT_ARR_1", "FLT_DEP_1"]].sum().reset_index()
totali["TOT_VOLI"] = totali["FLT_ARR_1"] + totali["FLT_DEP_1"]
top10 = totali.sort_values("TOT_VOLI", ascending=False).head(10)

# Riformatta per grafico
df_long = top10.melt(
    id_vars=["APT_NAME"],
    value_vars=["FLT_ARR_1", "FLT_DEP_1"],
    var_name="Tipo_Volo",
    value_name="Totale"
)
df_long["Tipo_Volo"] = df_long["Tipo_Volo"].replace({
    "FLT_ARR_1": "Arrivi",
    "FLT_DEP_1": "Partenze"
})

# Calcolo del range Y personalizzato
max_val = df_long["Totale"].max()
min_val = df_long["Totale"].min()
padding = (max_val - min_val) * 0.1  # aggiunge un po' di spazio

# Grafico a barre raggruppate con scala Y più stretta
fig = px.bar(
    df_long,
    x="APT_NAME",
    y="Totale",
    color="Tipo_Volo",
    barmode="group",
    title="Arrivi vs Partenze nei 10 aeroporti con più traffico",
    labels={"APT_NAME": "Aeroporto", "Totale": "Numero voli"}
)

fig.update_layout(
    yaxis=dict(range=[min_val - padding, max_val + padding]),
    xaxis_tickangle=30,
    margin=dict(t=70, b=120, l=60, r=40),
    height=600
)

fig.show()

In [41]:
import os
def crea_mappa_aeroporti():
    """
    Crea una mappa interattiva degli aeroporti presenti in flights.csv,
    usando le coordinate da airports.csv e Plotly Express.
    Ogni punto è proporzionale al totale voli (FLT_TOT_1) di entrata + uscita.
    """
    # 1. Percorsi ai file
    try:
        base_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        base_dir = os.getcwd()
    airports_path = os.path.join(base_dir, 'data', 'airports.csv')
    flights_path  = os.path.join(base_dir, 'data', 'flights.csv')
    for p in (airports_path, flights_path):
        if not os.path.isfile(p):
            raise FileNotFoundError(f"File non trovato: {p}")

    # 2. Lettura CSV
    airports_df = pd.read_csv(airports_path)
    flights_df  = pd.read_csv(flights_path)

    # 3. Raggruppa per aeroporto usando FLT_TOT_1 già presente
    totali = (
        flights_df
        .groupby(['APT_ICAO', 'APT_NAME'], as_index=False)['FLT_TOT_1']
        .sum()
        .rename(columns={'FLT_TOT_1': 'TOT_VOLI'})
    )

    # 4. Merge con coordinate da airports.csv
    mappa_data = (
        pd.merge(
            totali,
            airports_df[['ICAO','Latitude','Longitude','City','Country']],
            left_on='APT_ICAO',
            right_on='ICAO',
            how='left'
        )
        .dropna(subset=['Latitude','Longitude'])
    )

    if mappa_data.empty:
        print("❌ Nessun aeroporto con coordinate valide!")
        return None, mappa_data

    # 5. Crea scatter_mapbox: dimensione e colore proporzionali a TOT_VOLI
    fig = px.scatter_mapbox(
        mappa_data,
        lat='Latitude',
        lon='Longitude',
        size='TOT_VOLI',
        color='TOT_VOLI',
        color_continuous_scale=['Green','Red'],
        hover_name='APT_NAME',
        hover_data={
            'City': True,
            'Country': True,
            'TOT_VOLI': True
        },
        size_max=15,
        zoom=3,
        mapbox_style='open-street-map',
        height=700,
        title='📍 Mappa Aeroporti per Traffico Totale (entrata+uscita)'
    )
    fig.update_layout(
        margin={'l':0,'r':0,'t':50,'b':0},
        coloraxis_colorbar=dict(title="Totale voli", tickformat=",")
    )

    # 6. Mostra con scroll zoom e toolbar
    fig.show(config={
        'scrollZoom': True,
        'displayModeBar': True
    })

    # 7. Salvataggio HTML
    out_file = 'mappa_aeroporti_interattiva.html'
    fig.write_html(out_file)
    print(f"✅ Mappa salvata in: {out_file}")

    # 8. Statistiche e report
    min_v, max_v = mappa_data['TOT_VOLI'].min(), mappa_data['TOT_VOLI'].max()
    busiest    = mappa_data.loc[mappa_data['TOT_VOLI'].idxmax(), 'APT_NAME']
    least_busy = mappa_data.loc[mappa_data['TOT_VOLI'].idxmin(), 'APT_NAME']

    print("📊 Statistiche generali:")
    print(f"   • Aeroporti mappati: {len(mappa_data)}")
    print(f"   • Più trafficato:   {busiest} ({max_v:,} voli)")
    print(f"   • Meno trafficato:  {least_busy} ({min_v:,} voli)\n")

    print("🏆 Top 10 aeroporti per traffico totale:")
    top10 = mappa_data.nlargest(10, 'TOT_VOLI')[['APT_NAME','City','Country','TOT_VOLI']]
    for i, row in enumerate(top10.itertuples(), start=1):
        print(f"   {i:2d}. {row.APT_NAME} ({row.City}, {row.Country}) – {row.TOT_VOLI:,} voli")

    return fig, mappa_data

if __name__ == "__main__":
    crea_mappa_aeroporti()

C:\Users\Christian\AppData\Local\Temp\ipykernel_34076\378753514.py:48: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



✅ Mappa salvata in: mappa_aeroporti_interattiva.html
📊 Statistiche generali:
   • Aeroporti mappati: 326
   • Più trafficato:   Amsterdam - Schiphol (2,694,025 voli)
   • Meno trafficato:  Ventspils (82 voli)

🏆 Top 10 aeroporti per traffico totale:
    1. Amsterdam - Schiphol (Amsterdam, Netherlands) – 2,694,025 voli
    2. Paris-Charles-de-Gaulle (Paris, France) – 2,581,673 voli
    3. Frankfurt (Frankfurt, Germany) – 2,581,524 voli
    4. London - Heathrow (London, United Kingdom) – 2,446,474 voli
    5. Madrid - Barajas (Madrid, Spain) – 2,117,063 voli
    6. Munich (Munich, Germany) – 2,013,524 voli
    7. Barcelona (Barcelona, Spain) – 1,699,766 voli
    8. Istanbul Atatürk (Istanbul, Turkey) – 1,570,488 voli
    9. Rome - Fiumicino (Rome, Italy) – 1,518,100 voli
   10. Zürich (Zurich, Switzerland) – 1,372,321 voli
